In [1]:
from astropy.table import Table, vstack, unique, SortedArray
import glob
import numpy
import time
from datetime import date, timedelta, datetime
import psycopg2
import sqlite3
import pandas as pd
from functools import reduce
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
from desispec.coaddition import coadd

import matplotlib.pyplot as plt
import os
from desiutil.log import get_logger, DEBUG

In [2]:
#get array of yyyymmdds to loop through
#getUnprocessedDates.py
def getUnprocessedDates():
    start_time = time.time()
    # Connect to desi.db with POSTGRES to get latest observed and unprocessed yyyymmdd
    f = open('/global/cfs/cdirs/desi/science/td/secrets/desi_pg.txt') #what is more valid?
    file = f.read()
    db_name, db_user, db_pwd, db_host = file.split()
    conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_pwd, host=db_host)
    cur = conn.cursor()
    cur.execute("""SELECT DISTINCT yyyymmdd from fibermap_daily WHERE yyyymmdd > 20210604""") #most recent, remove in future
    desi_arr = cur.fetchall()
    cur.close()
    conn.close()
    
    # Open to transients_search.db for latest processed yyyymmdd to do comparison with unprocessed
    filename_conn = "/global/cfs/cdirs/desi/science/td/daily-search/transients_search.db"
    conn = sqlite3.connect(filename_conn)
    trans_arr = conn.execute("""SELECT DISTINCT yyyymmdd from unprocessed_exposures""").fetchall()
    conn.close()

    # Compare yyyymmdd from fibermap_daily with unprocessed_exposures, retaining those in fibermap_daily and not in unprocessed_daily
    night_arr = numpy.setdiff1d(desi_arr, trans_arr)

    print('len(night_arr): ' + str(len(night_arr)))
    print("--- get unprocessed dates took:  %s seconds ---" % (time.time() - start_time))
    
    return(night_arr) 

In [3]:
night_arr = getUnprocessedDates()
print(night_arr)

PermissionError: [Errno 13] Permission denied: '/global/cscratch1/sd/clepart/desi_pg.txt'

In [1]:
### for debugging purposes remove 'for a in night_arr:'-loop, assign yyyymmdd = 20210606 or 20210406 for example


for a in night_arr:   
    # read in and store in one place all the fibermap information in the spectra files
    yyyymmdd = a
    dats=[]
    for filename in glob.glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{yyyymmdd}/spectra-*.fits"):
        t = Table.read(filename, format='fits',hdu=1, memmap=True)[['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC', 'NIGHT']]
        t=t[t['OBJTYPE']=='TGT']
        dats.append(t)
    dats=vstack(dats, join_type='inner',metadata_conflicts='silent')
    # group all the observations by TARGET_RA and TARGET_DEC
    # note that this is more reliable than grouping by TARGETID as TARGETID is NOT a unique identifier of RA and DEC
    dats_group = dats.group_by(['TARGET_RA','TARGET_DEC'])
    
    # for each group make a tuple containing all tileid/petal combinations that have that ra/dec
    tile_petal=[]
    # for each group, determine the unique nights that occur
    night=[]
    for g in dats_group.groups:
        
        gu = unique(g, keys=['TILEID'])
        
        dum=[]
        for a,b,c in zip(gu['TILEID'],gu['PETAL_LOC'],gu['TARGETID']):
            dum.append((a,b,c))
        tile_petal.append(tuple(dum))


        du = unique(g, keys=['NIGHT'])
        
        for d in zip(du['NIGHT']):
            night.append((d))
        

    # compress things down to the unique tile/petal combinations
    tile_petal = list(set(tile_petal))
    # union sets that have intersecting tile/petal combinations
    for i in range(len(tile_petal)-1,0,-1):
        for j in range(i-1,-1,-1):
            if len(set(tile_petal[i]) & set(tile_petal[j])) !=0:
                tile_petal[j]=tile_petal[i]+tile_petal[j]
                del tile_petal[i]
                break
    # unique of each set
    for i in range(len(tile_petal)):
        tile_petal[i]=list(set(tile_petal[i]))
    
    unique_night = numpy.unique(night)
    
    for i in tile_petal:
        spectra = Spectra()
        dif_spectra = Spectra()
        ref_spectra = Spectra() ### or else NameError at 'if ref_spectra.num_spectra'... in the case of no reference nights
        for j in i:    ### covers case of multiple tile/petal pairs 
            filename =  f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{j[0]}/{yyyymmdd}/spectra-{j[1]}-{j[0]}-thru{yyyymmdd}.fits"
            spectra = read_spectra(filename)
            date = numpy.array(yyyymmdd)
            idx = numpy.in1d(unique_night, date)
            ref_night = unique_night[~idx]

            try:
                cur_spectra = (spectra.select(nights=date, targets = [j[2]]))
                ref_spectra = (spectra.select(nights=ref_night, targets = [j[2]]))
            except:
                pass ### this is the mechanism for eliminating targets that have no reference nights. 

            if ref_spectra.num_spectra() != 0:
            ### in the case of no ref nights, as 'pass' has allowed a reference spectra to be created despite a RuntimeError, this is the check that there is something to difference
                coadd(ref_spectra)
                dif_spectra.update(cur_spectra)
                coadd(dif_spectra)
                for b in dif_spectra.bands:
                    dif_spectra.flux[b][0] = cur_spectra.flux[b][0]-ref_spectra.flux[b][0]
                    ok = numpy.logical_and(dif_spectra.mask[b][0,:] == 0, dif_spectra.ivar[b][0] != 0)
                    plt.plot(cur_spectra.wave[b][ok], cur_spectra.flux[b][0,ok], color = 'red', label = "current spectra")
                    plt.plot(ref_spectra.wave[b][ok],ref_spectra.flux[b][0,ok], color = 'green', label = 'reference spectra')
                    plt.plot(dif_spectra.wave[b][ok],dif_spectra.flux[b][0,ok], color = 'cyan', label = 'differenced spectra')
                    if b == 'b':
                        plt.legend()
                #plt.show()
                
                # save each days candidate plots into one pdf
                with PdfPages('diff_candidates(%d).pdf' % yyyymmdd) as pdf:
                    fig = plt.figure()
                    pdf.savefig(fig)

    ### insert garbage collection potentially
            
    
    

NameError: name 'night_arr' is not defined